In [8]:
from matplotlib import rcParams
import matplotlib.pyplot as plt
import numpy as np

np.seterr(all='ignore')
rcParams['figure.figsize'] = (14, 6)

#扩展MOD
from funcat import * #FUNCAT的同花顺/通达信指标模块：和：&，或：|，其他基本相同。可以使用指标选股！！！
#from pytdx import * #通达信数据更新模块到QUANTAXIS的mongoDB
from tushare import * #TUSHARE数据连接更新。
from rqalpha import * #RQALPHA本地数据连接，通过 rqalpha update_bundle 8点后更新全部数据库
#from jaqs import * #QunatOS的jaqs客户端在线。
#from abupy import * #ABUPY本地化，数据在线下载更新，数据在线下载在根目录abu下
#from quantaxis import * #加载QUANTAXIS数据库模块，首先需要开启mongoDB+web服务器，数据更新通过通达信服务器。
#from hikyuu import * #HIKYUU本地化，数据库在根目录的stock。通过通达信导入。

#读取/设定数据库时间区域，当前股票，选股时间段。 
import time
import datetime
n1 = (time.strftime("20%y-%m-%d"))## 20yy-mm-dd格式，输出当日日期
print ("读取当日时间格式n1",n1) #当日时间格式：yyyy-mm-dd
n2 = (time.strftime("20%y%m%d"))## 20yymmdd格式，输出当日日期
print ("读取当日时间格式n2",n2) #当日时间格式：yyyymmdd
def getYesterday(): 
    today=datetime.date.today() 
    oneday=datetime.timedelta(days=1) 
    yesterday=today-oneday  
    return yesterday
n3 = (getYesterday())# 输出前一天日期
print ("读取前一日时间格式n3",n3) #前日时间格式：yyyy-mm-dd
i = datetime.datetime.now()#当前日期
n4 = ("%s%s%s" % (i.year, i.month, i.day) )
print ("读取当日时间格式n4",n4) #当日时间格式：yyyymmdd
n5 = ("%s" % i.isoformat() )
print ("读取当日时间格式n5",n5) #当日时间格式：yyyy-mm-dd:...
n8="20180101"#开始时间
print ("手动设定选股开始时间格式n8",n8)
n9="20180202"#结束时间
print ("手动设定选股结束时间格式n9",n9)
n10="20170101"#设定起始时间
print ("手动设定数据更新设定起始时间格式n10",n10)
x1="000001.XSHG"
print ("设定股票代码",x1)

#设定数据库连接：RQALPHA/TUSHARE
from funcat.data.tushare_backend import TushareDataBackend
from funcat.data.rqalpha_data_backend import RQAlphaDataBackend
backend = "tushare"
if backend == "rqalpha":
    set_data_backend(RQAlphaDataBackend("~/.rqalpha/bundle"))
elif backend == "tushare":
    set_data_backend(TushareDataBackend())
set_start_date(n10)

#当前股票最新数据显示。
S(x1)  # 设置当前关注股票
T(n3)   # 设置当前观察日期

print ("测试：打印 开盘 最高 最低 收盘 成交量",O, H, L, C, V)  # 打印 Open High Low Close
print ("当天涨幅",C / C[1] - 1 )  # 当天涨幅
print ("打印60日均线",MA(C, 60) )  # 打印60日均线
print ("判断收盘价是否大于60日均线",C > MA(C, 60))  # 判断收盘价是否大于60日均线
print ("30日最高价",HHV(H, 30))  # 30日最高价
print ("最近30日，收盘价 Close 大于60日均线的天数",COUNT(C > MA(C, 60), 30))  # 最近30日，收盘价 Close 大于60日均线的天数
print ("10日均线上穿",CROSS(MA(C, 10), MA(C, 20)))  # 10日均线上穿
print ("最近10天收阳线的天数",COUNT(C > O, 10))  # 最近10天收阳线的天数
print ("最近60天K线实体的最高价",HHV(MAX(O, C), 60))  # 最近60天K线实体的最高价
print ("5日均线上穿10日均线",CROSS(MA(C, 5), MA(C, 10)))  # 5日均线上穿10日均线


读取当日时间格式n1 2018-11-09
读取当日时间格式n2 20181109
读取前一日时间格式n3 2018-11-08
读取当日时间格式n4 2018119
读取当日时间格式n5 2018-11-09T13:12:52.410227
手动设定选股开始时间格式n8 20180101
手动设定选股结束时间格式n9 20180202
手动设定数据更新设定起始时间格式n10 20170101
设定股票代码 000001.XSHG
测试：打印 开盘 最高 最低 收盘 成交量 2660.09 2662.37 2632.3 2635.63 159635487.0
当天涨幅 -0.002161781519986028


FormulaException: only size-1 arrays can be converted to Python scalars

In [2]:
print ("《选股》读取时间：==》",("%s" % i.isoformat() )) 
# 选出涨停股
select(
    lambda : C / C[1] - 1 >= 0.0995,
    start_date=(n3),
	end_date=(n2),
)

# 选出最近30天K线实体最高价最低价差7%以内，最近100天K线实体最高价最低价差25%以内，
# 最近10天，收盘价大于60日均线的天数大于3天
select(
    lambda : ((HHV(MAX(C, O), 30) / LLV(MIN(C, O), 30) - 1 < 0.07)
              & (HHV(MAX(C, O), 100) / LLV(MIN(C, O), 100) - 1 > 0.25)
              & (COUNT(C > MA(C, 60), 10) > 3)
             ),
    start_date=(n3),
	end_date=(n2),
)

# 选出最近3天每天的成交量小于20日成交量均线，最近3天最低价低于20日均线，最高价高于20日均线
# 自定义选股回调函数
def callback(date, order_book_id, symbol):
    print("Cool, 在", date, "选出", order_book_id, symbol)    
select(
    lambda : (EVERY(V < MA(V, 20) / 2, 3) & EVERY(L < MA(C, 20), 3) & EVERY(H > MA(C, 20), 3)),
    start_date=(n3),
	end_date=(n2),
    callback=callback,
)


《选股》读取时间：==》 2018-11-09T10:37:44.205409
lambda : C / C[1] - 1 >= 0.0995,
[20181108]
20181108 000025.XSHE 000025.XSHE[特 力Ａ]
20181108 000509.XSHE 000509.XSHE[华塑控股]
20181108 000518.XSHE 000518.XSHE[四环生物]
20181108 000590.XSHE 000590.XSHE[启迪古汉]
20181108 000622.XSHE 000622.XSHE[恒立实业]
20181108 000638.XSHE 000638.XSHE[万方发展]
20181108 000691.XSHE 000691.XSHE[亚太实业]
20181108 000788.XSHE 000788.XSHE[北大医药]
20181108 000835.XSHE 000835.XSHE[长城动漫]
20181108 002052.XSHE 002052.XSHE[同洲电子]
20181108 002054.XSHE 002054.XSHE[德美化工]
20181108 002072.XSHE 002072.XSHE[凯瑞德]
20181108 002086.XSHE 002086.XSHE[东方海洋]
20181108 002103.XSHE 002103.XSHE[广博股份]
20181108 002112.XSHE 002112.XSHE[三变科技]
20181108 002192.XSHE 002192.XSHE[融捷股份]
20181108 002220.XSHE 002220.XSHE[天宝食品]
20181108 002328.XSHE 002328.XSHE[新朋股份]
20181108 002417.XSHE 002417.XSHE[深南股份]
20181108 002477.XSHE 002477.XSHE[雏鹰农牧]
20181108 002676.XSHE 002676.XSHE[顺威股份]
20181108 002708.XSHE 002708.XSHE[光洋股份]
20181108 002881.XSHE 002881.XSHE[美格智能]
20181108 002915.XSHE

In [3]:
print ("《结束选股》读取时间：==》",("%s" % i.isoformat() )) 

《结束选股》读取时间：==》 2018-02-06T10:56:35.054658
